# LlamaIndex + Gaugid Memory Block Integration

This notebook demonstrates how to use `GaugidMemoryBlock` with LlamaIndex's Memory system, allowing agents to store and retrieve long-term memories using Gaugid profiles.

**Installation**: `pip install gaugid[llama-index]`

Based on: https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/memory/memory.py

In [ ]:
import os
import asyncio

try:
    from llama_index.core.memory import Memory
    from llama_index.core.base.llms.types import ChatMessage
    from llama_index.llms.openai import OpenAI
    from gaugid.integrations.llama_index import GaugidMemoryBlock
    LLAMA_INDEX_AVAILABLE = True
except ImportError:
    LLAMA_INDEX_AVAILABLE = False
    print("⚠️  LlamaIndex not installed. Install with: pip install gaugid[llama-index]")

# Set your API keys
os.environ["OPENAI_API_KEY"] = "your-key-here"
os.environ["GAUGID_CONNECTION_TOKEN"] = "your-token-here"

In [ ]:
# Create Gaugid memory blocks
print("1️⃣ Creating GaugidMemoryBlocks...")

# User preferences block (semantic memory)
preferences_block = GaugidMemoryBlock(
    name="user_preferences",
    connection_token=os.getenv("GAUGID_CONNECTION_TOKEN"),
    description="User preferences and settings",
    priority=1,  # High priority - don't truncate
    memory_type="semantic",
)

# Knowledge block (semantic memory)
knowledge_block = GaugidMemoryBlock(
    name="user_knowledge",
    connection_token=os.getenv("GAUGID_CONNECTION_TOKEN"),
    description="User's knowledge and facts",
    priority=2,
    memory_type="semantic",
)

# Context block (episodic memory)
context_block = GaugidMemoryBlock(
    name="conversation_context",
    connection_token=os.getenv("GAUGID_CONNECTION_TOKEN"),
    description="Conversation context and history",
    priority=3,  # Lower priority - can be truncated
    memory_type="episodic",
)

print("   ✅ Created 3 memory blocks:")
print("      - user_preferences (semantic)")
print("      - user_knowledge (semantic)")
print("      - conversation_context (episodic)")

In [ ]:
# Create LlamaIndex Memory with Gaugid blocks
print("2️⃣ Creating LlamaIndex Memory with Gaugid blocks...")
memory = Memory(
    memory_blocks=[
        preferences_block,
        knowledge_block,
        context_block,
    ],
    session_id="user-123",
    token_limit=30000,
)
print("   ✅ Memory created")

In [ ]:
# Create LLM
llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4o-mini")

# Add messages to memory
print("3️⃣ Adding messages to memory...")
user_message = ChatMessage(
    role="user",
    content="I prefer dark mode for all applications."
)
await memory.aput(user_message)
print(f"   ✅ Added: {user_message.content}")

In [ ]:
# Add another message
user_message2 = ChatMessage(
    role="user",
    content="I'm working on a Python project using FastAPI."
)
await memory.aput(user_message2)
print(f"   ✅ Added: {user_message2.content}")